In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd 

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\ninja\AppData\Local\sagemaker\sagemaker\config.yaml


In [4]:
import os

os.environ['AWS_DEFAULT_REGION'] = 'eu-central-1'  # Replace with your desired region

In [5]:
import boto3
import sagemaker

# SageMaker ve S3 istemcilerini "eu-central-1" bölgesi için başlatma
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session(boto_session=sm_boto3)
region = sess.boto_session.region_name

bucket = "fall_detection"
print("Using bucket: " + bucket)


AttributeError: 'SageMaker' object has no attribute 'region_name'

In [2]:
import boto3
import sagemaker

# SageMaker istemcisini belirli bir bölgede ("eu-central-1") başlatma
boto_session = boto3.Session(region_name="eu-central-1")
sm_boto3 = boto_session.client("sagemaker")

# SageMaker oturumunu başlatma
sess = sagemaker.Session(boto_session=boto_session)

# Belirlenen bölgeyi doğrulama
region = boto_session.region_name
bucket = "falldetection123"
print("Using bucket: " + bucket + " in region: " + region)


Using bucket: falldetection123 in region: eu-central-1


In [3]:
print(region)

eu-central-1


In [4]:
df=pd.read_csv("data_imu_birlestirilmis_28_10.csv")

In [5]:
df.head()

,Unnamed: 0,ax,ay,az,gx,gy,gz,timestamp,time_diff,Class Name
0,0,-1.028198,7.406667,-2.838919,-0.418559,0.063694,0.027297,1900-01-01 17:55:40.747,NaN,not_falling
1,1,-1.020539,7.627713,-2.904742,-0.430347,0.050697,0.045961,1900-01-01 17:55:40.767,0 days 00:00:00.020000,not_falling
2,2,-1.013342,7.828984,-2.935427,-0.440230,0.041012,0.059841,1900-01-01 17:55:40.787,0 days 00:00:00.020000,not_falling
3,3,-1.003664,7.984988,-2.912928,-0.446552,0.034748,0.066260,1900-01-01 17:55:40.807,0 days 00:00:00.020000,not_falling
4,4,-0.980148,8.063047,-2.868723,-0.448961,0.024543,0.068392,1900-01-01 17:55:40.827,0 days 00:00:00.020000,not_falling


In [19]:
df.shape

(49736, 10)

In [6]:
df.isnull().sum()

Unnamed: 0    0
ax            0
ay            0
az            0
gx            0
gy            0
gz            0
timestamp     0
time_diff     4
Class Name    0
dtype: int64

In [21]:
df["Class Name"].value_counts()

Class Name
not_falling    40841
falling         8895
Name: count, dtype: int64

In [23]:
features=list(df.columns)

In [24]:
features

['Unnamed: 0',
 'ax',
 'ay',
 'az',
 'gx',
 'gy',
 'gz',
 'timestamp',
 'time_diff',
 'Class Name']

In [27]:
keep_columns = ['ax', 'ay', 'az', 'gx', 'gy', 'gz']

In [28]:
filtered_list = [item for item in features if item in keep_columns]

In [47]:
df = df.drop(columns=['Unnamed: 0', 'timestamp', 'time_diff'])

df

In [48]:
df

,ax,ay,az,gx,gy,gz,Class Name
0,-1.028198,7.406667,-2.838919,-0.418559,0.063694,0.027297,not_falling
1,-1.020539,7.627713,-2.904742,-0.430347,0.050697,0.045961,not_falling
2,-1.013342,7.828984,-2.935427,-0.440230,0.041012,0.059841,not_falling
3,-1.003664,7.984988,-2.912928,-0.446552,0.034748,0.066260,not_falling
4,-0.980148,8.063047,-2.868723,-0.448961,0.024543,0.068392,not_falling
...,...,...,...,...,...,...,...
49731,1.828079,9.423597,-0.645419,-1.292857,-0.038149,-2.056477,not_falling
49732,0.686294,9.158905,1.320737,-1.122093,-0.212877,-1.661709,not_falling
49733,1.196530,8.183237,1.745310,-0.915019,-0.535702,-1.189365,not_falling
49734,1.506319,8.012976,1.559735,-0.686853,-0.531130,-0.872752,not_falling


In [7]:
import numpy as np 

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
label_encoder = LabelEncoder()
df['Class Name'] = label_encoder.fit_transform(df['Class Name'])

In [10]:
window_size=50

In [11]:
def create_sequences(df, window_size):
    sequences = []
    labels = []

    for i in range(len(df) - window_size + 1):
        window = df.iloc[i:i + window_size]
        # En fazla hangi etiket varsa onu al
        label = window['Class Name'].mode()[0]  # Mod (en sık tekrar eden)
        sequence = window[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values
        sequences.append(sequence)
        labels.append(label)

    return np.array(sequences), np.array(labels)

# Pencereleri ve etiketleri oluşturma
sequences, labels = create_sequences(df, window_size)

# Sonuçları gösterme
print("Sequences shape:", sequences.shape)  # (num_sequences, 25, 6)
print("Labels shape:", labels.shape)        # (num_sequences,)
print("Example sequence:\n", sequences[0])  # İlk pencere örneği
print("Example label:", labels[0])           # İlk pencere etiket

Sequences shape: (49687, 50, 6)
Labels shape: (49687,)
Example sequence:
 [[-1.02819820e+00  7.40666667e+00 -2.83891892e+00 -4.18558559e-01
   6.36936937e-02  2.72972973e-02]
 [-1.02053856e+00  7.62771340e+00 -2.90474178e+00 -4.30347055e-01
   5.06972726e-02  4.59606268e-02]
 [-1.01334181e+00  7.82898366e+00 -2.93542709e+00 -4.40229801e-01
   4.10119749e-02  5.98410341e-02]
 [-1.00366358e+00  7.98498790e+00 -2.91292766e+00 -4.46551700e-01
   3.47484650e-02  6.62602790e-02]
 [-9.80148161e-01  8.06304741e+00 -2.86872307e+00 -4.48961444e-01
   2.45426599e-02  6.83921938e-02]
 [-9.40375457e-01  8.08909686e+00 -2.83576286e+00 -4.50772788e-01
   1.41129947e-02  6.80320919e-02]
 [-8.91046565e-01  8.10858096e+00 -2.83366679e+00 -4.55653963e-01
   1.07663127e-02  6.57920221e-02]
 [-8.69579312e-01  8.13848561e+00 -2.81770161e+00 -4.60877243e-01
   1.29503724e-02  6.10410561e-02]
 [-8.86708570e-01  8.17864988e+00 -2.75772214e+00 -4.63899297e-01
   1.62114262e-02  5.31561585e-02]
 [-8.79898274e-01

In [96]:
a,b=np.unique(labels,return_counts=True)

In [97]:
label_encoder.inverse_transform(a)

array(['falling', 'not_falling'], dtype=object)

In [98]:
b

array([ 9013, 40674], dtype=int64)

In [99]:
sequences[0] 

array([[-1.02819820e+00,  7.40666667e+00, -2.83891892e+00,
        -4.18558559e-01,  6.36936937e-02,  2.72972973e-02],
       [-1.02053856e+00,  7.62771340e+00, -2.90474178e+00,
        -4.30347055e-01,  5.06972726e-02,  4.59606268e-02],
       [-1.01334181e+00,  7.82898366e+00, -2.93542709e+00,
        -4.40229801e-01,  4.10119749e-02,  5.98410341e-02],
       [-1.00366358e+00,  7.98498790e+00, -2.91292766e+00,
        -4.46551700e-01,  3.47484650e-02,  6.62602790e-02],
       [-9.80148161e-01,  8.06304741e+00, -2.86872307e+00,
        -4.48961444e-01,  2.45426599e-02,  6.83921938e-02],
       [-9.40375457e-01,  8.08909686e+00, -2.83576286e+00,
        -4.50772788e-01,  1.41129947e-02,  6.80320919e-02],
       [-8.91046565e-01,  8.10858096e+00, -2.83366679e+00,
        -4.55653963e-01,  1.07663127e-02,  6.57920221e-02],
       [-8.69579312e-01,  8.13848561e+00, -2.81770161e+00,
        -4.60877243e-01,  1.29503724e-02,  6.10410561e-02],
       [-8.86708570e-01,  8.17864988e+00, -2.757

In [100]:
sequences[1] 

array([[-1.02053856e+00,  7.62771340e+00, -2.90474178e+00,
        -4.30347055e-01,  5.06972726e-02,  4.59606268e-02],
       [-1.01334181e+00,  7.82898366e+00, -2.93542709e+00,
        -4.40229801e-01,  4.10119749e-02,  5.98410341e-02],
       [-1.00366358e+00,  7.98498790e+00, -2.91292766e+00,
        -4.46551700e-01,  3.47484650e-02,  6.62602790e-02],
       [-9.80148161e-01,  8.06304741e+00, -2.86872307e+00,
        -4.48961444e-01,  2.45426599e-02,  6.83921938e-02],
       [-9.40375457e-01,  8.08909686e+00, -2.83576286e+00,
        -4.50772788e-01,  1.41129947e-02,  6.80320919e-02],
       [-8.91046565e-01,  8.10858096e+00, -2.83366679e+00,
        -4.55653963e-01,  1.07663127e-02,  6.57920221e-02],
       [-8.69579312e-01,  8.13848561e+00, -2.81770161e+00,
        -4.60877243e-01,  1.29503724e-02,  6.10410561e-02],
       [-8.86708570e-01,  8.17864988e+00, -2.75772214e+00,
        -4.63899297e-01,  1.62114262e-02,  5.31561585e-02],
       [-8.79898274e-01,  8.21734360e+00, -2.699

In [113]:
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.3,stratify=labels, random_state=12)

# Sonuçları gösterme
print("X_train shape:", X_train.shape)  #  
print("X_test shape:", X_test.shape)      #  
print("y_train shape:", y_train.shape)    #  
print("y_test shape:", y_test.shape)      # 

X_train shape: (34780, 50, 6)
X_test shape: (14907, 50, 6)
y_train shape: (34780,)
y_test shape: (14907,)


In [102]:
import pickle

In [103]:
data_to_save = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test
}

 
with open('data_arrays.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

print("NumPy arrays have been saved to data_arrays.pkl")

NumPy arrays have been saved to data_arrays.pkl


In [104]:
 
with open('data_arrays.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

X_train_loaded = loaded_data['X_train']
X_test_loaded = loaded_data['X_test']
y_train_loaded = loaded_data['y_train']
y_test_loaded = loaded_data['y_test']
 
print("Loaded X_train shape:", X_train_loaded.shape)
print("Loaded X_test shape:", X_test_loaded.shape)
print("Loaded y_train shape:", y_train_loaded.shape)
print("Loaded y_test shape:", y_test_loaded.shape)


Loaded X_train shape: (34780, 50, 6)
Loaded X_test shape: (14907, 50, 6)
Loaded y_train shape: (34780,)
Loaded y_test shape: (14907,)


In [105]:
y_train

array([1, 0, 1, ..., 0, 1, 1])

In [106]:
# Eğitim verilerini pickle dosyasına kaydetme
train_data_to_save = {
    'X_train': X_train,
    'y_train': y_train
}

with open('train_pickle.pkl', 'wb') as f:
    pickle.dump(train_data_to_save, f)

print("NumPy arrays have been saved to train_pickle.pkl")

# Test verilerini pickle dosyasına kaydetme
test_data_to_save = {
    'X_test': X_test,
    'y_test': y_test
}

with open('test_pickle.pkl', 'wb') as f:
    pickle.dump(test_data_to_save, f)

print("NumPy arrays have been saved to test_pickle.pkl")


NumPy arrays have been saved to train_pickle.pkl
NumPy arrays have been saved to test_pickle.pkl


In [107]:
# Eğitim verilerini yükleme
with open('train_pickle.pkl', 'rb') as f:
    loaded_train_data = pickle.load(f)

X_train_loaded = loaded_train_data['X_train']
y_train_loaded = loaded_train_data['y_train']

# Test verilerini yükleme
with open('test_pickle.pkl', 'rb') as f:
    loaded_test_data = pickle.load(f)

X_test_loaded = loaded_test_data['X_test']
y_test_loaded = loaded_test_data['y_test']


In [8]:
bucket

'falldetection123'

In [21]:
sk_prefix

'sagemaker/fall_detection_classification/sklearncontainer'

In [ ]:

#sending data to s3 , 

sk_prefix="sagemaker/fall_detection_classification/sklearncontainer"
trainpath=sess.upload_data(path="train_pickle.pkl",bucket=bucket,key_prefix=sk_prefix)
testpath=sess.upload_data(path="test_pickle.pkl",bucket=bucket,key_prefix=sk_prefix)

print(trainpath)
print(testpath)



s3://falldetection123/sagemaker/fall_detection_classification/sklearncontainer/train_pickle.pkl
s3://falldetection123/sagemaker/fall_detection_classification/sklearncontainer/test_pickle.pkl


In [13]:
import pickle

In [14]:
 
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)   
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)     

print("X_train reshaped shape:", X_train_reshaped.shape)  
print("X_test reshaped shape:", X_test_reshaped.shape)    


NameError: name 'X_train' is not defined

In [ ]:
 
train_data_to_save = {
    'X_train': X_train_reshaped,
    'y_train': y_train
}

with open('train_pickle_random_forest.pkl', 'wb') as f:
    pickle.dump(train_data_to_save, f)

print("NumPy arrays have been saved to train_pickle.pkl")

 
test_data_to_save = {
    'X_test': X_test_reshaped,
    'y_test': y_test
}

with open('test_pickle_random_forest.pkl', 'wb') as f:
    pickle.dump(test_data_to_save, f)

print("NumPy arrays have been saved to test_pickle.pkl")


NumPy arrays have been saved to train_pickle.pkl
NumPy arrays have been saved to test_pickle.pkl


In [15]:
 
with open('train_pickle_random_forest.pkl', 'rb') as f:
    loaded_train_data = pickle.load(f)

X_train_loaded = loaded_train_data['X_train']
y_train_loaded = loaded_train_data['y_train']
 
with open('test_pickle_random_forest.pkl', 'rb') as f:
    loaded_test_data = pickle.load(f)

X_test_loaded = loaded_test_data['X_test']
y_test_loaded = loaded_test_data['y_test']


In [ ]:
# s3 e veri gönderme kodu 

sk_prefix="sagemaker/fall_detection_classification/sklearncontainer"
trainpath=sess.upload_data(path="train_pickle_random_forest.pkl",bucket=bucket,key_prefix=sk_prefix)
testpath=sess.upload_data(path="test_pickle_random_forest.pkl",bucket=bucket,key_prefix=sk_prefix)

print(trainpath)
print(testpath)



s3://falldetection123/sagemaker/fall_detection_classification/sklearncontainer/train_pickle_random_forest.pkl
s3://falldetection123/sagemaker/fall_detection_classification/sklearncontainer/test_pickle_random_forest.pkl


In [16]:
trainpath="s3://falldetection123/sagemaker/fall_detection_classification/sklearncontainer/train_pickle_random_forest.pkl"
testpath="s3://falldetection123/sagemaker/fall_detection_classification/sklearncontainer/test_pickle_random_forest.pkl"
sk_prefix="sagemaker/fall_detection_classification/sklearncontainer"

In [18]:
train_df=pd.read_pickle('train_pickle_random_forest.pkl')
test_df=pd.read_pickle("test_pickle_random_forest.pkl")

In [19]:
train_df

{'X_train': array([[ 0.68831886,  8.20594994, -4.15952752, ...,  0.28683116,
         -0.16130135,  0.14843587],
        [ 4.48217772,  6.5658369 ,  2.49315504, ...,  0.31313517,
         -0.07944006, -2.60086926],
        [-7.45277452, -5.00710093,  2.90725887, ...,  1.13913248,
         -0.85724757,  1.34758232],
        ...,
        [ 0.28997896, 11.80627377,  0.61132388, ...,  1.86740118,
         -0.40607173,  0.60889271],
        [-3.94371725,  8.16725242, -2.53464654, ...,  0.68377823,
         -0.52337796,  0.40838449],
        [-2.71635289,  8.34202028, -3.0484175 , ..., -0.86280443,
          1.54610758, -1.19308458]]),
 'y_train': array([1, 0, 1, ..., 0, 1, 1])}

In [20]:
test_df["y_test"]

array([1, 1, 1, ..., 1, 1, 0])

In [21]:
train_df

{'X_train': array([[ 0.68831886,  8.20594994, -4.15952752, ...,  0.28683116,
         -0.16130135,  0.14843587],
        [ 4.48217772,  6.5658369 ,  2.49315504, ...,  0.31313517,
         -0.07944006, -2.60086926],
        [-7.45277452, -5.00710093,  2.90725887, ...,  1.13913248,
         -0.85724757,  1.34758232],
        ...,
        [ 0.28997896, 11.80627377,  0.61132388, ...,  1.86740118,
         -0.40607173,  0.60889271],
        [-3.94371725,  8.16725242, -2.53464654, ...,  0.68377823,
         -0.52337796,  0.40838449],
        [-2.71635289,  8.34202028, -3.0484175 , ..., -0.86280443,
          1.54610758, -1.19308458]]),
 'y_train': array([1, 0, 1, ..., 0, 1, 1])}

%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd 


def model_fn(model_dir):
    model_path = os.path.join(model_dir, "model.joblib")
    if os.path.exists(model_path):
        clf = joblib.load(model_path)
        return clf
    else:
        raise FileNotFoundError(f"Model file not found at {model_path}")



def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        input_data = pd.DataFrame(json.loads(request_body))
        return input_data
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")

def predict_fn(input_data, model):
    predictions = model.predict(input_data)
    return predictions

def output_fn(predictions, content_type):
    if content_type == 'application/json':
        return json.dumps(predictions.tolist())
    else:
        raise ValueError(f"Unsupported content type: {content_type}")


if __name__=="__main__":

    print(" extracting arugments")
    parser=argparse.ArgumentParser()

    # hyperparameter
    parser.add_argument("--n_estimators",type=int,default=3)
    parser.add_argument("--random_state",type=int,default=0)
    

    # data,model and outpu _directories 

    parser.add_argument("--model-dir",type=str,default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train",type=str,default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("-test",type=str,default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file",type=str,default="train_pickle_random_forest.pkl")
    parser.add_argument("--test-file",type=str,default="test_pickle_random_forest.pkl")


    args,_=parser.parse_known_args()

    print("SKLearn version: ",sklearn.__version__)
    print("joblib version ",joblib.__version__)

    train_df=pd.read_pickle(os.path.join(args.train,args.train_file))
    test_df=pd.read_pickle(os.path.join(args.test,args.test_file))

    X_train=train_df["X_train"]
    X_test=test_df["X_test"]
    y_train=train_df["y_train"]
    y_test=test_df["y_test"]

    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)

    print("training random forest model ")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=2, n_jobs=1)

    model.fit(X_train,y_train)
    print()
    model_path=os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model,model_path)

    y_pred_test=model.predict(X_test)
    test_acc=accuracy_score(y_test,y_pred_test)
    test_rep=classification_report(y_test,y_pred_test)

    print("test accuracy",test_acc)
    print("test report",test_rep)






In [108]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import sklearn
import joblib
import argparse
import os
import numpy as np
import pandas as pd
import json

# Model yükleme fonksiyonu
def model_fn(model_dir):
    model_path = os.path.join(model_dir, "model.joblib")
    if os.path.exists(model_path):
        clf = joblib.load(model_path)
        return clf
    else:
        raise FileNotFoundError(f"Model file not found at {model_path}")

# Tahmin için gerekli input_fn, predict_fn ve output_fn fonksiyonlar

if __name__ == "__main__":

    print("Extracting arguments")
    parser = argparse.ArgumentParser()

    # Hiperparametreler
    parser.add_argument("--n_estimators", type=int, default=3)
    parser.add_argument("--random_state", type=int, default=0)

    # Veri, model ve çıktı dizinleri
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train_pickle_random_forest.pkl")
    parser.add_argument("--test-file", type=str, default="test_pickle_random_forest.pkl")

    args, _ = parser.parse_known_args()

    print("SKLearn version: ", sklearn.__version__)
    print("joblib version ", joblib.__version__)

    # Eğitim ve test verilerini yükleme
    train_df = pd.read_pickle(os.path.join(args.train, args.train_file))
    test_df = pd.read_pickle(os.path.join(args.test, args.test_file))

    X_train = train_df["X_train"]
    X_test = test_df["X_test"]
    y_train = train_df["y_train"]
    y_test = test_df["y_test"]

    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)

    print("Training RandomForest model")
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=2, n_jobs=1)
    model.fit(X_train, y_train)

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print(f"Model saved at {model_path}")

    # Test set üzerinde tahmin yapma ve değerlendirme
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_report = classification_report(y_test, y_pred_test)

    print("Test accuracy:", test_acc)
    print("Test report:", test_report)


Overwriting script.py


In [109]:
import os

os.environ['AWS_DEFAULT_REGION'] = 'eu-central-1'  # Replace with your desired region


In [23]:
train_df=pd.read_pickle("train_pickle_random_forest.pkl")
test_df=pd.read_pickle("test_pickle_random_forest.pkl")

X_train=train_df["X_train"]
X_test=test_df["X_test"]
y_train=train_df["y_train"]
y_test=test_df["y_test"]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(34780, 300)
(14907, 300)
(34780,)
(14907,)


In [41]:
X_test[0].shape

(300,)

In [25]:
X_train.shape

(34780, 300)

In [26]:
X_test.shape

(14907, 300)

In [110]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [111]:
model = RandomForestClassifier(n_estimators=1, max_depth=10, random_state=42)

# Modeli eğitme
model.fit(X_train, y_train)

# Test seti ile tahmin yapma
y_pred = model.predict(X_test)

# Model değerlendirme
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Doğruluk (Accuracy):", accuracy)
print("Kesinlik (Precision):", precision)
print("Duyarlılık (Recall):", recall)
print("F1 Skoru:", f1)

Doğruluk (Accuracy): 0.8885758368551687
Kesinlik (Precision): 0.9081616849930308
Duyarlılık (Recall): 0.9610751454560355
F1 Skoru: 0.933869490783135


In [33]:
y_pred_0 = model.predict(X_test[0].reshape(1,-1)) 
print(y_pred_0)

[1]


In [112]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::692859911732:role/sagemakeraccess",
    instance_count=1,
    instance_type="ml.m5.large",  # Doğru parametre
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 1,
        "random_state": 0
    },
    use_spot_instance=True,
    max_run=3600
)


In [113]:
region

'eu-central-1'

# training model 

In [114]:
sklearn_estimator.fit({"train":trainpath,"test":testpath},wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-11-06-19-55-11-041


2024-11-06 19:55:12 Starting - Starting the training job...
2024-11-06 19:55:26 Starting - Preparing the instances for training...
2024-11-06 19:55:52 Downloading - Downloading input data...
2024-11-06 19:56:22 Downloading - Downloading the training image..2024-11-06 19:57:03,118 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-06 19:57:03,123 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-06 19:57:03,180 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-06 19:57:03,417 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-06 19:57:03,438 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-06 19:57:03,460 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-06 19:57:03,477 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "ad

2024-11-01 13:23:55 Starting - Starting the training job...
2024-11-01 13:24:09 Starting - Preparing the instances for training...
2024-11-01 13:24:36 Downloading - Downloading input data...
2024-11-01 13:25:06 Downloading - Downloading the training image...
2024-11-01 13:25:48 Training - Training image download completed. Training in progress.2024-11-01 13:25:52,256 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-01 13:25:52,260 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-01 13:25:52,308 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-01 13:25:52,461 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-01 13:25:52,473 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-01 13:25:52,485 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-01 13:25:52,494 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_sklearn_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "n_estimators": 100,
        "random_state": 0
    },
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "test": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated",
            "RecordWrapperType": "None"
        },
        "train": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated",
            "RecordWrapperType": "None"
        }
    },
    "input_dir": "/opt/ml/input",
    "is_master": true,
    "job_name": "RF-custom-sklearn-2024-11-01-13-23-52-786",
    "log_level": 20,
    "master_hostname": "algo-1",
    "model_dir": "/opt/ml/model",
    "module_dir": "s3://sagemaker-eu-central-1-692859911732/RF-custom-sklearn-2024-11-01-13-23-52-786/source/sourcedir.tar.gz",
    "module_name": "script",
    "network_interface_name": "eth0",
    "num_cpus": 2,
    "num_gpus": 0,
    "output_data_dir": "/opt/ml/output/data",
    "output_dir": "/opt/ml/output",
    "output_intermediate_dir": "/opt/ml/output/intermediate",
    "resource_config": {
        "current_host": "algo-1",
        "current_instance_type": "ml.m5.large",
        "current_group_name": "homogeneousCluster",
        "hosts": [
            "algo-1"
        ],
        "instance_groups": [
            {
                "instance_group_name": "homogeneousCluster",
                "instance_type": "ml.m5.large",
                "hosts": [
                    "algo-1"
                ]
            }
        ],
        "network_interface_name": "eth0"
    },
    "user_entry_point": "script.py"
}
Environment variables:
SM_HOSTS=["algo-1"]
SM_NETWORK_INTERFACE_NAME=eth0
SM_HPS={"n_estimators":100,"random_state":0}
SM_USER_ENTRY_POINT=script.py
SM_FRAMEWORK_PARAMS={}
SM_RESOURCE_CONFIG={"current_group_name":"homogeneousCluster","current_host":"algo-1","current_instance_type":"ml.m5.large","hosts":["algo-1"],"instance_groups":[{"hosts":["algo-1"],"instance_group_name":"homogeneousCluster","instance_type":"ml.m5.large"}],"network_interface_name":"eth0"}
SM_INPUT_DATA_CONFIG={"test":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"},"train":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"}}
SM_OUTPUT_DATA_DIR=/opt/ml/output/data
SM_CHANNELS=["test","train"]
SM_CURRENT_HOST=algo-1
SM_MODULE_NAME=script
SM_LOG_LEVEL=20
SM_FRAMEWORK_MODULE=sagemaker_sklearn_container.training:main
SM_INPUT_DIR=/opt/ml/input
SM_INPUT_CONFIG_DIR=/opt/ml/input/config
SM_OUTPUT_DIR=/opt/ml/output
SM_NUM_CPUS=2
SM_NUM_GPUS=0
SM_MODEL_DIR=/opt/ml/model
SM_MODULE_DIR=s3://sagemaker-eu-central-1-692859911732/RF-custom-sklearn-2024-11-01-13-23-52-786/source/sourcedir.tar.gz
SM_TRAINING_ENV={"additional_framework_parameters":{},"channel_input_dirs":{"test":"/opt/ml/input/data/test","train":"/opt/ml/input/data/train"},"current_host":"algo-1","framework_module":"sagemaker_sklearn_container.training:main","hosts":["algo-1"],"hyperparameters":{"n_estimators":100,"random_state":0},"input_config_dir":"/opt/ml/input/config","input_data_config":{"test":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"},"train":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"}},"input_dir":"/opt/ml/input","is_master":true,"job_name":"RF-custom-sklearn-2024-11-01-13-23-52-786","log_level":20,"master_hostname":"algo-1","model_dir":"/opt/ml/model","module_dir":"s3://sagemaker-eu-central-1-692859911732/RF-custom-sklearn-2024-11-01-13-23-52-786/source/sourcedir.tar.gz","module_name":"script","network_interface_name":"eth0","num_cpus":2,"num_gpus":0,"output_data_dir":"/opt/ml/output/data","output_dir":"/opt/ml/output","output_intermediate_dir":"/opt/ml/output/intermediate","resource_config":{"current_group_name":"homogeneousCluster","current_host":"algo-1","current_instance_type":"ml.m5.large","hosts":["algo-1"],"instance_groups":[{"hosts":["algo-1"],"instance_group_name":"homogeneousCluster","instance_type":"ml.m5.large"}],"network_interface_name":"eth0"},"user_entry_point":"script.py"}
SM_USER_ARGS=["--n_estimators","100","--random_state","0"]
SM_OUTPUT_INTERMEDIATE_DIR=/opt/ml/output/intermediate
SM_CHANNEL_TEST=/opt/ml/input/data/test
SM_CHANNEL_TRAIN=/opt/ml/input/data/train
SM_HP_N_ESTIMATORS=100
SM_HP_RANDOM_STATE=0
PYTHONPATH=/opt/ml/code:/miniconda3/bin:/miniconda3/lib/python37.zip:/miniconda3/lib/python3.7:/miniconda3/lib/python3.7/lib-dynload:/miniconda3/lib/python3.7/site-packages
Invoking script with the following command:
/miniconda3/bin/python script.py --n_estimators 100 --random_state 0
 extracting arugments
SKLearn version:  0.23.2
joblib version  1.2.0
(34780, 300)
(14907, 300)
(34780,)
(14907,)
training random forest model 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 100
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
test accuracy 0.9823572818139129
test report               precision    recall  f1-score   support
           0       0.99      0.91      0.95      2704
           1       0.98      1.00      0.99     12203
    accuracy                           0.98     14907
   macro avg       0.98      0.96      0.97     14907
weighted avg       0.98      0.98      0.98     14907
2024-11-01 13:27:33,565 sagemaker-containers INFO     Reporting training SUCCESS

2024-11-01 13:27:52 Uploading - Uploading generated training model
2024-11-01 13:27:52 Completed - Training job completed
Training seconds: 195
Billable seconds: 195


In [164]:
artifact

NameError: name 'artifact' is not defined

In [115]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact=sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]


2024-11-06 19:57:21 Starting - Preparing the instances for training
2024-11-06 19:57:21 Downloading - Downloading the training image
2024-11-06 19:57:21 Training - Training image download completed. Training in progress.
2024-11-06 19:57:21 Uploading - Uploading generated training model
2024-11-06 19:57:21 Completed - Training job completed


In [116]:
artifact

's3://sagemaker-eu-central-1-692859911732/RF-custom-sklearn-2024-11-06-19-55-11-041/output/model.tar.gz'

In [117]:
artifact

's3://sagemaker-eu-central-1-692859911732/RF-custom-sklearn-2024-11-06-19-55-11-041/output/model.tar.gz'

## deployin model to endpoints

 # sagemaker endpoint

In [118]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime,strftime

model_name="Custom-sklearn-model-"+strftime("%Y-%m-%d-%H-%M-%S",gmtime())
model=SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::692859911732:role/sagemakeraccess",
    entry_point= "script.py",
    framework_version=FRAMEWORK_VERSION

)

In [119]:
model

# hali hazırda endpoint varsa 

In [103]:
import boto3
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import json
import numpy as np

# Mevcut endpoint adınızı buraya girin
endpoint_name = "Custom-sklearn-model2024-11-06-19-32-14"

# Predictor nesnesini mevcut endpoint ile bağlayın
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),   # Veriyi JSON formatına dönüştürür
    deserializer=JSONDeserializer() # Yanıtı JSON formatında okur
)

# endpoint deployment

In [120]:
endpoint_name="Custom-sklearn-model"+strftime("%Y-%m-%d-%H-%M-%S",gmtime())
print("endpointname={}".format(endpoint_name))

predictor=model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)

endpointname=Custom-sklearn-model2024-11-06-19-59-03


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2024-11-06-19-58-53
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model2024-11-06-19-59-03
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model2024-11-06-19-59-03


------!

In [123]:
predictor.predict(X_test[1].reshape(1,-1))

array([1])

In [99]:
predictor

In [59]:
input_data.shape

(300,)

In [61]:
X_test[0].shape

(300,)

In [97]:
predictor.predict(X_test[2].reshape(1,-1))

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: [[-2.37400599e+00  8.90073719e+00 -2.80147131e+00  1.66390317e-01
   3.14164065e-01  4.64268421e-01 -2.37558692e+00  8.78504635e+00
  -2.87306057e+00  1.22825721e-01  2.73812707e-01  4.16514401e-01
  -2.39788006e+00  8.69900375e+00 -2.90004231e+00  8.53934528e-02
   2.30666002e-01  3.75378531e-01 -2.44431930e+00  8.62412028e+00
  -2.83468301e+00  4.97295887e-02  1.90066366e-01  3.39597549e-01
  -2.49064397e+00  8.52009487e+00 -2.85200594e+00 -5.39866859e-03
   1.46722931e-01  2.91033870e-01 -2.52112270e+00  8.40443582e+00
  -2.92867872e+00 -6.81562296e-02  1.08063920e-01  2.41309861e-01
  -2.53914913e+00  8.34057968e+00 -2.88199964e+00 -1.04410883e-01
   8.53104808e-02  2.22869079e-01 -2.55250679e+00  8.31342853e+00
  -2.83827462e+00 -1.33664767e-01  5.89760976e-02  2.20495258e-01
  -2.56003225e+00  8.29985706e+00 -2.85211382e+00 -1.69283817e-01
   2.57837335e-02  2.17030958e-01 -2.55099584e+00  

In [87]:
X_test[2].reshape(1,-1).shape[1] == 300

True

In [89]:
X_test[2].reshape(1,-1).shape[1]

300

In [92]:
input_data=X_test[2]
import json

In [93]:
input_data = np.array(json.loads(input_data))

TypeError: the JSON object must be str, bytes or bytearray, not ndarray

In [95]:
import numpy as np
import json
from sagemaker.predictor import Predictor

# Örneğin, 3 örnek ve her örnekte 300 özellik içeren rastgele bir veri oluşturma
data = np.random.rand(3, 300).tolist()

# JSON formatında veri hazırlama
payload = json.dumps(data)

# Predictor ile gönderim

response = predictor.predict(payload)
print(response)


ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint sagemaker-endpoint-name of account 692859911732 not found.

In [99]:
import json

In [100]:
input_data = np.frombuffer(input_data, dtype=np.float32)

In [101]:
input_data.shape

(600,)

In [ ]:
input_data = input_data.reshape(1, -1)

In [104]:
input_data.shape

(1, 600)

In [105]:
def input_fn(request_body, request_content_type):
    # Check if the input data is in JSON format
    if request_content_type == 'application/json':
        try:
            # Convert JSON-formatted string into a numpy array
            input_data = np.array(json.loads(request_body))
            
            # Ensure the input data has the right shape (1, 300)
            if input_data.shape != (1, 300):
                input_data = input_data.reshape(1, 300)
            
            return input_data
        
        except (ValueError, TypeError) as e:
            raise ValueError(f"Error processing JSON input: {e}")
    
    # Check if the input data is in .npy format
    elif request_content_type == 'application/x-npy':
        try:
            # Load binary data directly as a numpy array
            input_data = np.frombuffer(request_body, dtype=np.float32)
            
            # Ensure the input data has the right shape (1, 300)
            if input_data.size != 300:
                raise ValueError("The .npy input data does not have 300 features.")
            input_data = input_data.reshape(1, 300)
            
            return input_data
        
        except Exception as e:
            raise ValueError(f"Error processing .npy input: {e}")
    
    else:
        # Raise an error if the content type is unsupported
        raise ValueError(f"Unsupported content type: {request_content_type}")

In [ ]:
def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        # JSON formatında gelen veriyi numpy array'e dönüştür
        input_data = np.array(json.loads(request_body))
        return input_data
    elif request_content_type == 'application/x-npy':
        # .npy formatındaki veriyi numpy array olarak yükle
        input_data = np.frombuffer(request_body, dtype=np.float32)  # Veri türünü modelin beklediği türe göre ayarlayın
        input_data = input_data.reshape(1, -1)  # Modelin beklediği boyuta göre şekillendir
        return input_data  # DataFrame'e dönüştürmeye gerek yok, direk numpy array olarak döndürüyoruz
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")

In [107]:
sample_input = np.random.rand(1, 300)  # Random data with shape (1, 300)

# Convert sample input to JSON format
input_data = sample_input.tolist()  # Convert numpy array to list for JSON serialization

# Send the input to the model
response = predictor.predict(input_data)

# Print the prediction result
print("Prediction:", response)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://eu-central-1.console.aws.amazon.com/cloudwatch/home?region=eu-central-1#logEventViewer:group=/aws/sagemaker/Endpoints/Custom-sklearn-model2024-11-06-09-40-54 in account 692859911732 for more information.

In [108]:
import numpy as np

# Assuming `input_data` is your input numpy array for prediction
input_data = np.nan_to_num(input_data, nan=0.0, posinf=0.0, neginf=0.0)


In [109]:
# Clip values to be within the range of float32
input_data = np.clip(input_data, -3.4e38, 3.4e38)


In [110]:
import json
import numpy as np
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# Example cleaned input data
input_data = np.random.rand(1, 300) * 1e6  # Adjusting values to simulate large input
input_data = np.nan_to_num(input_data, nan=0.0, posinf=0.0, neginf=0.0)
input_data = np.clip(input_data, -3.4e38, 3.4e38)
 
response = predictor.predict(input_data.tolist())
print("Prediction:", response)


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://eu-central-1.console.aws.amazon.com/cloudwatch/home?region=eu-central-1#logEventViewer:group=/aws/sagemaker/Endpoints/Custom-sklearn-model2024-11-06-09-40-54 in account 692859911732 for more information.

In [83]:
import json
import time
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
import paho.mqtt.client as mqtt
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# SageMaker endpoint ayarları

class RealTimeProcessor:
    def __init__(self, desired_freq=32, interpolation_freq=50):
        self.desired_freq = desired_freq   
        self.interpolation_freq = interpolation_freq / 1000   
        self.data_buffer = []
        self.filtered_data = []
        self.interpolated_data_list = []

    def select_32_samples(self, data_list):
        data_list = sorted(data_list, key=lambda x: x['timestamp'])
        df = pd.DataFrame(data_list)
        df = df.drop_duplicates(subset=['timestamp'])  
        if len(df) >= self.desired_freq:
            indices = np.linspace(0, len(df) - 1, self.desired_freq, dtype=int)
            selected_data = df.iloc[indices].to_dict('records')
        else:
            selected_data = df.to_dict('records')
        return selected_data

    def process_data(self):
        selected_data = self.select_32_samples(self.data_buffer)
        df = pd.DataFrame(selected_data)
        df = df.sort_values(by='timestamp')
        
        self.filtered_data = self.apply_ekf(df)
        self.interpolated_data = self.interpolate_data(self.filtered_data)

        self.predict_data(self.interpolated_data)
        self.data_buffer.clear()

    def apply_ekf(self, df):
        filtered_data = []
        for index, row in df.iterrows():
            z = np.array([row['ax'], row['ay'], row['az'], row['gx'], row['gy'], row['gz']])
            x_est = self.ekf_filter(z)
            filtered_data.append({
                'timestamp': row['timestamp'],
                'ax_filtered': x_est[0],
                'ay_filtered': x_est[1],
                'az_filtered': x_est[2],
                'gx_filtered': x_est[3],
                'gy_filtered': x_est[4],
                'gz_filtered': x_est[5]
            })
        return pd.DataFrame(filtered_data)

    def ekf_filter(self, z):
        x_est = z  
        return x_est

    def interpolate_data(self, df):
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        timestamps_unix = df['timestamp'].astype(np.int64) // 10**6  
        values = df[['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered']].values
        new_timestamps_unix = np.linspace(timestamps_unix.min(), timestamps_unix.max(), 50)
    
        try:
            cs = CubicSpline(timestamps_unix, values, axis=0, bc_type='natural')
            new_values = cs(new_timestamps_unix)
        except Exception as e:
            print(f"Interpolasyon hatası: {e}")
            return pd.DataFrame()
    
        interpolated_df = pd.DataFrame(new_values, columns=['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered'])
        interpolated_df['timestamp'] = pd.to_datetime(new_timestamps_unix, unit='ms')
    
        print("\nInterpolasyon Sonucu (50 Hz):")
        print(interpolated_df.head())
    
        self.interpolated_data_list.append(interpolated_df)
    
        return interpolated_df

    def predict_data(self, df):
        if len(df) >= 50:
            features = df[['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered']].values.flatten().reshape(1, -1)
            payload = json.dumps(features.tolist())  # Veriyi JSON formatına dönüştür
            
            try:
                # SageMaker endpoint'e tahmin talebinde bulun
                response = predictor.predict(payload)
                prediction = response['predictions'] if 'predictions' in response else response
                print("Gerçek Zamanlı Tahmin Sonucu:", prediction[0])
                
            except Exception as e:
                print("Tahmin yaparken hata oluştu:", e)
        else:
            print("50 örnek toplanamadı, tahmin yapılmıyor.")
            print(len(df))

    def add_data(self, data):
        self.data_buffer.append(data)
        print(f"Toplanan veri sayısı: {len(self.data_buffer)}")  
        if len(self.data_buffer) >= self.desired_freq:
            self.process_data()
        
        time.sleep(0.03)

mqtt_broker = "192.168.108.135"
mqtt_port = 1883
topic = "esp32_4/data"
processor = RealTimeProcessor()

def on_message(client, userdata, message):
    payload = message.payload.decode('utf-8')
    data = json.loads(payload)
    data['timestamp'] = time.time() * 1000  
    processor.add_data(data)

client = mqtt.Client()
client.on_message = on_message
client.connect(mqtt_broker, mqtt_port, 60)
client.subscribe(topic)
client.loop_start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    client.loop_stop()
    client.disconnect()


C:\Users\ninja\AppData\Local\Temp\ipykernel_46236\3480009841.py:122: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Toplanan veri sayısı: 1
Toplanan veri sayısı: 2
Toplanan veri sayısı: 3
Toplanan veri sayısı: 4
Toplanan veri sayısı: 5
Toplanan veri sayısı: 6
Toplanan veri sayısı: 7
Toplanan veri sayısı: 8
Toplanan veri sayısı: 9
Toplanan veri sayısı: 10
Toplanan veri sayısı: 11
Toplanan veri sayısı: 12
Toplanan veri sayısı: 13
Toplanan veri sayısı: 14
Toplanan veri sayısı: 15
Toplanan veri sayısı: 16
Toplanan veri sayısı: 17
Toplanan veri sayısı: 18
Toplanan veri sayısı: 19
Toplanan veri sayısı: 20
Toplanan veri sayısı: 21
Toplanan veri sayısı: 22
Toplanan veri sayısı: 23
Toplanan veri sayısı: 24
Toplanan veri sayısı: 25
Toplanan veri sayısı: 26
Toplanan veri sayısı: 27
Toplanan veri sayısı: 28
Toplanan veri sayısı: 29
Toplanan veri sayısı: 30
Toplanan veri sayısı: 31
Toplanan veri sayısı: 32

Interpolasyon Sonucu (50 Hz):
   ax_filtered  ay_filtered  az_filtered  gx_filtered  gy_filtered  \
0    -1.050000    -0.720000    10.040000        -0.07          0.0   
1    -1.065746    -0.725913    10.1204

In [85]:
import numpy as np
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

 
# Test verisi (örneğin, modelin beklediği şekil ile eşleştiğinden emin olun)
test_data = np.random.rand(1, 300).tolist()  # Örneğin, 1x300 boyutunda veri

# Tahmin yapın
try:
    response = predictor.predict(test_data)
    print("Tahmin Sonucu:", response)
except Exception as e:
    print("Tahmin yaparken hata oluştu:", e)


Tahmin yaparken hata oluştu: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://eu-central-1.console.aws.amazon.com/cloudwatch/home?region=eu-central-1#logEventViewer:group=/aws/sagemaker/Endpoints/Custom-sklearn-model2024-11-05-21-35-35 in account 692859911732 for more information.


In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import numpy as np

# Endpoint adını belirleyin
endpoint_name = "Custom-sklearn-model2024-11-01-15-19-23"

# Predictor nesnesini oluşturun
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),   # Veriyi JSON formatına dönüştürür
    deserializer=JSONDeserializer() # Yanıtı JSON formatında okur
)

# Giriş verisi (Örneğin, modelin beklediği özellik sayısına göre 300 adet değer)
input_data = np.random.rand(1, 300).tolist()  # 1x300 boyutunda örnek veri

# Tahmin çağrısı yapın
result = predictor.predict(input_data)

# Sonucu yazdırın
print("Tahmin Sonucu:", result)


X=df[filtered_list]
y=df["Class Name"]

y.head()

X.head()

# wındow sequence 

In [ ]:
df

# modeli burada eğitip sagemaker ile sadece dağıtım yapabiliriz :

In [ ]:
import joblib

# Modeli eğitme (zaten yaptığınız adımları içeriyor)
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

# Eğitilmiş modeli kaydetme
joblib.dump(model, "model.joblib")


In [ ]:
import boto3
import sagemaker
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

# SageMaker oturumunu başlatma
sagemaker_session = sagemaker.Session()
role = "YOUR_SAGEMAKER_ROLE"  # SageMaker IAM rolünüzü belirtin

# Model dosyasını S3'e yükleyin
bucket = sagemaker_session.default_bucket()
prefix = 'random-forest-endpoint'
model_path = f"s3://{bucket}/{prefix}/model.joblib"
sagemaker_session.upload_data("model.joblib", bucket=bucket, key_prefix=prefix)

# Endpoint için model objesini oluşturun
sklearn_model = SKLearnModel(
    model_data=model_path,
    role=role,
    entry_point="script.py",  # Modeli yüklemek ve tahmin yapmak için gerekli kodları içeren script
    framework_version="0.23-1"  # sklearn versiyonu
)

# Endpoint adı
endpoint_name = "random-forest-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# Endpoint'i oluşturma
predictor = sklearn_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name
)


In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import numpy as np

predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

# Tahmin için örnek veri seçme
sample_data = X_test.iloc[0].tolist()  # 300 özellikli bir örnek

# JSON formatına dönüştürüp tahmin istemi gönderme
result = predictor.predict([sample_data])
print("Tahmin Sonucu:", result)
